In [ ]:
import re
import numpy as np
from scripts.scrape import get_data, get_test_data



# Day 15

In [ ]:
data = get_data(15)
test = get_test_data(15)
data

In [ ]:
data = [int(d) for d in data[0].split(',')]

In [ ]:
seq = data.copy()

last = {}
for i in range(len(seq)):
    last[seq[i]] = [i, i]
for i in range(len(seq), 2020):
    lookup = seq[i-1]
    val = last[lookup][1] - last[lookup][0]
    seq.append(val)
    if val in last.keys():
        last[val] = [last[val][1], i]
    else: 
        last[val] = [i, i]        
seq[-1]

In [ ]:
# Part 2

seq = data.copy()

last = {}
for i in range(len(seq)):
    last[seq[i]] = [i, i]
for i in range(len(seq), 30000000):
    lookup = seq[i-1]
    val = last[lookup][1] - last[lookup][0]
    seq.append(val)
    if val in last.keys():
        last[val] = [last[val][1], i]
    else: 
        last[val] = [i, i]        

seq[-1]